In [1]:
import time

now = int(time.time())

In [2]:
from pathlib import Path

import pandas as pd

data_dir = Path().cwd() / "data"
dataset_dir = data_dir / "dataset"

csv_train = pd.read_csv(dataset_dir / "train.csv")
csv_test = pd.read_csv(dataset_dir / "test.csv")
csv_clothing_master = pd.read_csv(dataset_dir / "clothing_master.csv")

In [3]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()

for col in ["Division Name", "Department Name", "Class Name"]:
    df_encoded = pd.get_dummies(
        csv_clothing_master[col].astype(str),
        prefix=col,
    ).astype(int)
    csv_clothing_master = csv_clothing_master.drop(col, axis=1)
    csv_clothing_master = pd.concat([csv_clothing_master, df_encoded], axis=1)


def preprocess(x: pd.DataFrame) -> pd.DataFrame:
    x = x.copy()
    x = x.merge(csv_clothing_master, on="Clothing ID")
    x = x.drop(
        [
            "Clothing ID",
            "Title",
            "Review Text",
        ],
        axis=1,
    )
    cols = [
        "Age",
        "Positive Feedback Count",
    ]
    x[cols] = scaler.fit_transform(x[cols])
    return x


train = preprocess(csv_train)
test = preprocess(csv_test)

train_x, train_y = (
    train.drop(["Recommended IND", "Rating"], axis=1),
    train["Recommended IND"],
)


In [4]:
import lightning as pl
import torch
from torch.utils import data

pl.seed_everything(42, workers=True)

train_val = data.TensorDataset(
    torch.tensor(train_x.values, dtype=torch.float32),
    torch.tensor(train_y, dtype=torch.float32),
)

train_size = int(len(train_val) * 0.9)
val_size = len(train_val) - train_size
dataset_train, dataset_val = data.random_split(
    train_val,
    lengths=[train_size, val_size],
)
dataset_test = data.TensorDataset(torch.tensor(test.values, dtype=torch.float32))

Seed set to 42


In [5]:
from torch import nn, optim
from torch.optim import optimizer


class LightningModel(pl.LightningModule):
    def __init__(self, model: nn.Module) -> None:
        super().__init__()
        self.model = model
        self.loss_fn = nn.functionapl.mse_loss
        self.save_hyperparameters()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    def configure_optimizers(self) -> optimizer.Optimizer:
        return optim.Adam(self.parameters(), lr=1e-3)

    def training_step(self, batch: list[torch.Tensor]) -> torch.Tensor:
        inputs, target = batch
        output = self(inputs)
        loss = self.loss_fn(output, target)
        self.log(
            "train_loss",
            loss,
            prog_bar=True,
            sync_dist=True,
            on_step=True,
            on_epoch=False,
        )
        return loss

    def validation_step(self, batch: list[torch.Tensor]) -> torch.Tensor:
        inputs, target = batch
        output = self(inputs)
        loss = self.loss_fn(output, target)
        self.log(
            "val_loss",
            loss,
            prog_bar=True,
            sync_dist=True,
            on_step=True,
            on_epoch=True,
        )
        return loss

    def predict_step(self, batch: list[torch.Tensor]) -> torch.Tensor:
        inputs, *_ = batch
        return self(inputs)


In [6]:
from lightning.pytorch.loggers import WandbLogger

import wandb

wandb.login()
logger = WandbLogger(name=f"atmaCup17_{now}", project="atmaCup17", log_model="all")

model = LightningModel(
    nn.Sequential(nn.Linear(len(train_x.columns), 64), nn.ReLU(), nn.Linear(64, 1)),
)

trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=50,
    log_every_n_steps=20,
    devices=1,
    logger=logger,
)

batch_size = 32
num_workers = 2
trainer.fit(
    model,
    train_dataloaders=data.DataLoader(
        dataset_train,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
    ),
    val_dataloaders=data.DataLoader(
        dataset_val,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
    ),
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
/home/kira/ghq/github.com/ras0q/atmaCup17/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 1.9 K  | train
---------------------------------------------
1.9 K     Trainable params
0         Non-trainable params
1.9 K     Total params
0.007     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_336103/2363724382.py:35: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = self.loss_fn(output, target)


Epoch 0:   1%|          | 2/282 [00:00<00:15, 18.32it/s, v_num=u3w4, train_loss=0.986]

...

Epoch 0: 100%|██████████| 282/282 [00:01<00:00, 161.00it/s, v_num=u3w4, train_loss=0.191, val_loss_step=0.114, val_loss_epoch=0.146]

/tmp/ipykernel_336103/2363724382.py:35: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = self.loss_fn(output, target)


Epoch 199: 100%|██████████| 282/282 [00:01<00:00, 143.99it/s, v_num=u3w4, train_loss=0.192, val_loss_step=0.112, val_loss_epoch=0.145] 

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 282/282 [00:02<00:00, 131.23it/s, v_num=u3w4, train_loss=0.192, val_loss_step=0.112, val_loss_epoch=0.145]


In [7]:
outputs = trainer.predict(
    model,
    data.DataLoader(
        dataset_test,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
    ),
)
if outputs is None:
    msg = "none output"
    raise ValueError(msg)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 349/349 [00:00<00:00, 452.30it/s]


In [8]:
import csv

import numpy as np

with (data_dir / f"submit_{now}.csv").open("w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["target"])
    writer.writerows([[np.mean(x.tolist())] for x in outputs])